importing main libraries

In [1]:
from tkinter import *
from tkinter.ttk import *
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import cv2
import numpy as np

filebrowsing function and seat belt detecting 

In [2]:

#open filebrowser 
def filebrowser():
    filename = filedialog.askopenfilename(initialdir='/', title="select image ",
                                          filetypes=(("image","*.png*"),
                                                     ("AllFiles","*.*")))
    
# Load image
    img = cv2.imread(filename)

# Resize image
    img = cv2.resize(img, (260, 510))

# Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Apply Canny edge detection
    edges = cv2.Canny(img, 55, 165, apertureSize=3, L2gradient=False )
# Apply Hough Transform
    lines = cv2.HoughLinesP(edges, rho=1, theta=np.pi/180, threshold=50, minLineLength=50, maxLineGap=20)
# Initialize variables
    left_line = None
    right_line = None

# Loop through lines
    for line in lines:
        x1, y1, x2, y2 = line[0] 
        #making shure x2 - x1 =! 0
        if x2 - x1 == 0:
            slope = 9999
        else:
            slope = (y2 - y1) / (x2 - x1)

    # Filter out lines that are too short or vertical
        if abs(slope) < 0.5 or abs(slope) > 2:
            continue

    # Separate lines into left and right
        if slope < 0:
            if right_line is None or x1 > right_line[0][2]:
                right_line = line
        else:
            if left_line is None or x1 < left_line[0][0]:
                left_line = line
            
            

# Calculate the intersection of the two lines
    if left_line is not None and right_line is not None:
        left_x1, left_y1, left_x2, left_y2 = left_line[0]
        right_x1, right_y1, right_x2, right_y2 = right_line[0]

        left_slope = (left_y2 - left_y1) / (left_x2 - left_x1)
        left_intercept = left_y1 - left_slope * left_x1

        right_slope = (right_y2 - right_y1) / (right_x2 - right_x1)
        right_intercept = right_y1 - right_slope * right_x1

        x = int((right_intercept - left_intercept) / (left_slope - right_slope))
        y = int(left_slope * x + left_intercept)
    # drawing rectangle over the seat belt 
        cv2.rectangle(img, (right_x1, right_y1), (right_x2, right_y2), (0, 0, 255), 2)
        cv2.rectangle(img, (x, y), (right_x2, right_y1), (0, 0, 255 ), 2)
        
        open_result()

    else:
        seatbeltoff()

# Display image
    cv2.imshow("Seat Belt Detection", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


creating main window

In [3]:

window1 = Tk()
window1.geometry("500x500")
window1.title("seat belt")
window1.config(background="#AFD3E2")

creating bill window 

In [4]:
# if seat belt is on
def open_result():
    global label
    #creating window 
    on = tk.Toplevel(window1)
    on.geometry("300x300")
    on.title("seat belt is on ")
    on.config(background="#AFD3E2")
#writing info 
    name = Label(on,
                  text = "name : Reem ", font = '14',foreground = 'black',  background="#AFD3E2")
    name.place(x = 10, y = 40)
    carnum = Label(on,
                  text = "carnum : 1111 | ر ق م", font = '14',foreground = 'black',  background="#AFD3E2")
    carnum.place(x = 10, y = 80)
    fine = Label(on,
                  text = "traffic fine: Non ", font = '14',foreground = 'black',  background="#AFD3E2")
    fine.place(x = 10, y = 120)
    #exit buttom 
    button1 = Button(on, text = "Exit",
                     command = on.destroy)
    button1.place(x = 80, y = 180)
    return on
#if seat belt is off
def seatbeltoff():
    
    #creating window 
    off = Toplevel()

    off.title("seatbelt is off")

    off.geometry("300x300")
    off.config(background="#080011")
#writing info 
    name = Label(off,
                text = "name : Karim  ", font = '14',foreground = 'black',  background="#AFD3E2")
    name.place(x = 10, y = 40)
    carnum = Label(off,
                text = "carnum : 1111 | ر ق م", font = '14',foreground = 'black',  background="#AFD3E2")
    carnum.place(x = 10, y = 80)
    fine = Label(off,
                text = "traffic fine: 2000 EGP ", font = '14',foreground = 'black',  background="#AFD3E2")
    fine.place(x = 10, y = 120)
     #exit buttom 
    
    button = Button(off, text = "Exit",
                command = off.destroy)

    button.place(x = 80, y = 180)

    off.mainloop()


main window 

In [5]:
#creating stile object 
style = Style()
style.configure('TButton', font =
               ('BentonSans', 15, 'bold'),
                    borderwidth = '8' )

style.map('TButton', foreground = [('active', 'red')],
                     background = [('active', 'black')])

photo = PhotoImage(file = r"pngwing.com (1).png")

photoimage = photo.subsample(10, 10)


uploadimage = Button(window1,
                     command=filebrowser,
                     text='Upload Image', image = photoimage, compound = LEFT )

uploadimage.place(x = 120, y = 180, width = 260, height = 100)

window1.mainloop()